In [15]:
!pip install torch torchvision pillow

In [37]:
# Import necessary libraries
import os
import torch  # PyTorch library for deep learning
import torch.nn as nn  # Neural network module in PyTorch
from torchvision import models, transforms  # Pre-trained models and image transformations
from PIL import Image  # Library for image processing

# Dictionary to map class indices to building names
class_to_building = {
    0: "Butler Hall",
    1: "Carpenter Hall",
    2: "Lee Hall",
    3: "McCain Hall",
    4: "McCool Hall",
    5: "Old Main",
    6: "Simrall Hall",
    7: "Student Union",
    8: "Swalm Hall",
    9: "Walker Hall"
}

# Define the device for computation (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to load a pre-trained model for inference
def load_model(num_classes, model_path="/content/best_model.pth"):
    # Load a ResNet50 model pre-trained on ImageNet
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

    # Replace the fully connected layer to match the number of classes in our dataset
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # Load the trained model weights
    model.load_state_dict(torch.load(model_path, map_location=device), strict=False)

    # Move the model to the appropriate device and set it to evaluation mode
    model.to(device)
    model.eval()
    return model

# Load the model, specifying the number of classes and the path to the saved weights
num_classes = 10  # Replace with the actual number of classes in the dataset
model = load_model(num_classes, "/content/best_model.pth")

# Function to preprocess an image for model inference
def preprocess_image(image_path, image_size=512):
    # Define transformations: resize, convert to tensor, and normalize
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Mean and std for ImageNet
    ])

    # Open and preprocess the image, converting to RGB format
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  # Add batch dimension for model input
    return image.to(device)

# Function to predict the building type given an image
def predict(model, image_path, class_to_building):
    # Preprocess the image
    image = preprocess_image(image_path)

    # Make prediction without computing gradients (inference mode)
    with torch.no_grad():
        outputs = model(image)  # Get model outputs
        probabilities = torch.softmax(outputs, dim=1)  # Convert outputs to probabilities
        _, predicted = outputs.max(1)  # Get the class with the highest probability

    # Get predicted class index and confidence score
    class_idx = predicted.item()
    confidence = probabilities[0, class_idx].item()

    # Map the class index to the building name
    building_name = class_to_building.get(class_idx, "Unknown Building")

    # Print the prediction and confidence level
    print(f"Predicted Building: {building_name}")
    print(f"Confidence: {confidence:.4f}")

    return building_name, confidence

# New function to predict all images in a folder
def predict_folder(model, folder_path, class_to_building):
    # Loop through each file in the folder
    for image_file in os.listdir(folder_path):
        # Construct the full image path
        image_path = os.path.join(folder_path, image_file)

        # Check if the file is an image
        if image_path.endswith(('.jpg', '.jpeg', '.png')):
            # Predict and print results for each image
            building_name, confidence = predict(model, image_path, class_to_building)
            print(f"File: {image_file}, Predicted Building: {building_name}, Confidence: {confidence:.4f}")




<ipython-input-37-d14726b7781d>:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device), strict=False)


In [39]:
# Path to your test images folder
test_folder = "/content/test_images"  # Replace with the path to your folder in Colab

# Run prediction on all images in the folder
predict_folder(model, test_folder, class_to_building)


Predicted Building: Lee Hall
Confidence: 0.9938
File: building.jpeg, Predicted Building: Lee Hall, Confidence: 0.9938
Predicted Building: Lee Hall
Confidence: 0.7370
File: building2.jpg, Predicted Building: Lee Hall, Confidence: 0.7370
